## 1. 自注意力机制的学习

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$

- Q：查询矩阵, 表示当前处理的输入元素
- K：键矩阵, 表示输入元素的特征元素
- V：值矩阵, 是么个输入元素所有包含的信息, 当计算出与查询的匹配程度（通过键）后，最终需要提取活聚合的信息
- dk：键向量的维度
- QKT：查询与键的点积
- softmax：用于将点积结果归一化为概率分布

In [79]:
import numpy as np

# 输入序列
tokens = [
    "The", "cat", "sat", "on", "the", "mat"
]


Q = K = V =[
    [0.1, 0.2, 0.3, 0.4], # The
    [0.5, 0.6, 0.7, 0.8], # cat
    [0.9, 1.0, 1.1, 1.2], # sat
    [1.3, 1.4, 1.5, 1.6], # on
    [1.7, 1.8, 1.9, 2.0], # the
    [2.1, 2.2, 2.3, 2.4]  # mat  
]

Q = np.array(Q)
K = np.array(K)
V = np.array(V)

# Q[1]与 K 的每行做点积
scores = np.dot(Q[1], K.T)
print("scores", scores)


sqrt_dk = np.sqrt(K.shape[1])
print('sqrt_dk',sqrt_dk)

scale_scores = scores / sqrt_dk
print('scale_scores',scale_scores)


def softmax(x):
    # 将一组实数转换为0-1 之间的概率分布
    return np.exp(x) / np.sum(np.exp(x), axis=0)

weights = softmax(scale_scores)
print('weights',weights)

output = np.dot(weights, V)
print('output',output)
# 这就是 cat 注意力机制的最终输出

scores [0.7  1.74 2.78 3.82 4.86 5.9 ]
sqrt_dk 2.0
scale_scores [0.35 0.87 1.39 1.91 2.43 2.95]
weights [0.0315077  0.05299682 0.08914212 0.14993952 0.25220241 0.42421143]
output [1.62438656 1.72438656 1.82438656 1.92438656]


## 2. 多头注意力

多头注意力通过将查询、键、值向量分割成多个较小的矩阵，称为头，然后分别进行自注意力的计算，最后将所有头的输出合并起来。
#### 优势：
1. 捕获词元之间的不同语义和语法关系。
2. 增加模型的容量和表达性
3. 提高模型并行化和效率

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O
$$

其中，每个注意力头 $\text{head}_i$ 的计算公式为：

$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

注意力函数 $\text{Attention}(Q, K, V)$ 使用缩放点积注意力计算：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

## 符号说明
- $W_i^Q$,  $W_i^K$ , $W_i^V$: 用于第i个注意力头的查询、键和值的权重矩阵。
- $W^O$: 用于将所有注意力头输出连接后的权重矩阵。
- $h$: 注意力头的数量。
- $d_k$: 键向量的维度。
- $text{Concat}$: 将所有头的输出进行连接操作。

# 3. 位置编码

# Positional Encoding in Transformer

在Transformer模型中，位置编码用于在输入序列中注入位置信息，使模型能够利用序列中元素的相对和绝对位置。

## 公式

位置编码向量 $\text{PE}_{(pos, 2i)}$ 和 $\text{PE}_{(pos, 2i+1)}$ 的计算公式为：

$$
\text{PE}_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

$$
\text{PE}_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

其中：

- $pos$ 是序列中位置的索引。
- $i$ 是位置编码中的维度索引。
- $d_{\text{model}}$ 是模型的维度。

## 解释

- **偶数维度**：对于位置 $pos$ 的偶数维度 $2i$，使用正弦函数。
- **奇数维度**：对于位置 $pos$ 的奇数维度 $2i+1$，使用余弦函数。
- **缩放因子**：缩放因子 $10000^{\frac{2i}{d_{\text{model}}}}$ 确保不同维度的频率变化。

这种编码方式使得不同位置的编码在向量空间中彼此不同，同时允许模型通过加法保留位置信息。

In [92]:
# 输入序列
tokens = [
    "The", "cat", "sat", "on", "the", "mat"
]


Q = K = V =[
    [0.1, 0.2, 0.3, 0.4], # The
    [0.5, 0.6, 0.7, 0.8], # cat
    [0.9, 1.0, 1.1, 1.2], # sat
    [1.3, 1.4, 1.5, 1.6], # on
    [1.7, 1.8, 1.9, 2.0], # the
    [2.1, 2.2, 2.3, 2.4]  # mat  
]

position = [0, 1, 2, 3, 4, 5]
d_model = 4

# 使用正弦函数来计算位置编码
def positional_encoding(position, d_model):
    """
    生成位置编码矩阵。

    参数:
    - max_len: 序列的最大长度。
    - d_model: 模型的维度。

    返回:
    - pos_encoding: 形状为 (max_len, d_model) 的位置编码矩阵。
    """
    # 创建位置编码矩阵，初始化为零
    pos_encoding = np.zeros((len(position), d_model))

    # 计算每个位置和每个维度的值
    for pos in range(len(position)):
        for i in range(0, d_model, 2):
            angle = pos / np.power(10000, (2 * i) / np.float32(d_model))
            pos_encoding[pos, i] = np.sin(angle)
            if i + 1 < d_model:
                f = i + 1
                pos_encoding[pos, i + 1] = np.cos(angle)
                print(f)

    return pos_encoding


input_vector = positional_encoding(position, d_model)
print('*********input vector*********\n',input_vector)

# 把位置向量嵌入到输入向量中
Q = Q + input_vector
print('*********after add position vector*********\n',Q)
# 这些输入随后被送入 transformer 模型的编码器中

1
3
1
3
1
3
1
3
1
3
1
3
*********input vector*********
 [[ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 8.41470985e-01  5.40302306e-01  9.99999998e-05  9.99999995e-01]
 [ 9.09297427e-01 -4.16146837e-01  1.99999999e-04  9.99999980e-01]
 [ 1.41120008e-01 -9.89992497e-01  2.99999995e-04  9.99999955e-01]
 [-7.56802495e-01 -6.53643621e-01  3.99999989e-04  9.99999920e-01]
 [-9.58924275e-01  2.83662185e-01  4.99999979e-04  9.99999875e-01]]
*********after add position vector*********
 [[0.1        1.2        0.3        1.4       ]
 [1.34147098 1.14030231 0.7001     1.8       ]
 [1.80929743 0.58385316 1.1002     2.19999998]
 [1.44112001 0.4100075  1.5003     2.59999996]
 [0.9431975  1.14635638 1.9004     2.99999992]
 [1.14107573 2.48366219 2.3005     3.39999988]]


In [ ]:
d_model = 3
for i in range(0, d_model, 2):
    print(i)
    if i + 1 < d_model:
        print(i)